In [1]:
# each instance of this class stores a single fittness rule
class Fitness_Rule :  
    # takes a string which containes the rule and builds the rule
    def __init__(self, line) :
        self.option_list = []
        line_list = line.split('#')
        for element in line_list :
            if element.count(':') == 0 :
                self.option_list.append(element)
            else :
                self.option_list.append(element[:element.find(':')])
                self.fitnes_value = float(element[element.find(':')+2:])    
                
    # print overload
    def __str__(self) :
        out = ""
        for element in self.option_list :
            out += element
            out += " ** "
        out = out[:-3]
        out += " : " + str(self.fitnes_value)
        return out
        
    # takes an option dictionariy and returns the cost of this rule
    def get_partial_fitness(self, option_activation) :
        applicable = 1
        for it in self.option_list :
            applicable = applicable and option_activation[it]
        return applicable * self.fitnes_value        

    
# contains all fittness rules
class Fitness_Modell :    
    
    # takes a list containing fittness rules and builds the rule list
    def __init__(self) :
        self.fittness_rule_list = []
        
        
    # parse names to ints to use them as an index later
    def parse_names_to_index(self, dimacs_input) : 
        
        name_index_dict = dict()
        
        dimacs_input = [x.strip() for x in dimacs_input]
        for line in dimacs_input :
            # fill constraint list 
            if len(line) > 0 and line[0] == 'c' :
                line_list = line.split(' ')
                if len(line_list) == 3 :
                    name_index_dict[line_list[2]] = int(line_list[1].strip('$'))
        
        for rule in self.fittness_rule_list :
            for i in range(0, len(rule.option_list)) :
                if not rule.option_list[i] == 'root' :
                    rule.option_list[i] = name_index_dict[rule.option_list[i]]
                else :
                    rule.option_list[i] = 0
       # print(name_index_dict)
       # for it in self.fittness_rule_list :
        #    print(it)
    
    # takes a string containing fittness rules and adds them to the rule list
    def add_fitness_rules(self, fittness_input) :
        for line in fittness_input :
            self.fittness_rule_list.append(Fitness_Rule(line))        

                        
    # print overload
    def __str__(self) :
        out = ""
        for element in self.fittness_rule_list :
            out += element.__str__() + "\n"
        out = out[:-2]
        return out
    
    # takes an option activation dictionary and calculates the fitness(or cost) value
    def calculate_fitness(self, option_activation) : 
        fittness = 0.0
        for element in self.fittness_rule_list :
            fittness += element.get_partial_fitness(option_activation)
        return fittness



# takes a dictionary and re
#def calculate fitness()

In [2]:
# an instance of this class stores all constrains
# TODO: optimize validity check
class Constraint_Model :
        
    def __init__(self) :
        self.constraint_list = []
        self.simplified_constraint_list = []
        self.global_tabu_list = []
        self.mandatory_activation = []
                 
    # input line(string) list
    def build_model(self, dimacs_input) :
        self.constraint_list = []
        self.global_tabu_list = []
        self.mandatory_activation = []
        tmp_constraint = []
        
        dimacs_input = [x.strip() for x in dimacs_input]
        for line in dimacs_input :
            # fill constraint list 
            if len(line) > 0 and line[0] != 'c' and line[0] != 'p' :
                line_list = line.split(' ')
                for element in line_list :
                    if element == '0' and len(element) > 0 :
                        self.constraint_list.append(tmp_constraint)
                        tmp_constraint = []
                    elif len(element) > 0 :
                        tmp_constraint.append(int(element))      
            # initialize tabu list and mandatory activation
            elif len(line) > 0 and line[0] == 'p' :
                line_list = line.split(' ')
                self.global_tabu_list = [False] * (int(line_list[2]) + 1)
                self.mandatory_activation = [None] * (int(line_list[2]) + 1)

        self.simplified_constraint_list = self.constraint_list
                
                
    # print function            
    def __str__ (self) :
        out = "Constraints:\n"
        for a in self.constraint_list :
            out += str(a) + "-"+ str(len(a)) + ", "
        
        out += "\n\nGLobal Tabu List:\n"
        out += str(self.global_tabu_list)
        
        out += "\n\nMandatory Activation List:\n"
        out += str(self.mandatory_activation)
           
        return out
    
    def get_constraints(self) :
        out = []
        for a in self.constraint_list :
            out.append(a)
        return out
        
    
    # TO DO: delete more useless constraints
    # builds the global tabu list and sets the activation values 
    def build_global_tabu_list(self) :
        new_constraint_list = []
        for constraint in self.constraint_list :  
            if len(constraint) == 1 :       
                self.global_tabu_list[abs(constraint[0])] = True      
                self.mandatory_activation[abs(constraint[0])] = constraint[0] > 0    
            else :
                new_constraint_list.append(constraint)
                
        self.simplified_constraint_list = new_constraint_list
    
    
    # TODO: optimize !!!!!!
    def get_violated_variables (self, activation_arry) :
        violated_variables = []
        for constraint in self.simplified_constraint_list :
            if not self.check_partial_validity(constraint, activation_arry) :
                for v in constraint :
                    if abs(v) not in violated_variables :
                        violated_variables.append(abs(v))
        violated_variables.sort()
        return violated_variables
    
    
    # returns the first violated constraint
    def get_violated_constraint(self, activation_arry) :
        for constraint in self.simplified_constraint_list :
            if not self.check_partial_validity(constraint, activation_arry) :
                return constraint
        return []   
    
    # TODO: optimize !!!!!!    
    def check_validity (self, activation_arry) :
        for constraint in self.simplified_constraint_list :
            if not self.check_partial_validity(constraint, activation_arry) :
                return False
        return True
    
    # TODO: optimize
    def check_partial_validity (self, constraint, activation_arry) :
        for x in constraint :
            if x > 0 :
                if activation_arry[x] :
                    return True
            else :
                 if not activation_arry[x * (-1)] :
                    return True   
        return False    

In [3]:
def linear_regression(n) :
    import itertools
    combinations = list(map(list, itertools.product([0, 1], repeat=n)))
    return(combinations)

def parse_xml_to_dimacs(file):
    var_map = [] #

    final_string = "" #
    variables_string = "" #
    p_line = "p cnf " #
    clauses = "" #
    current_options = [] # 
    optional_false_elements = []
    clauses_2D = []

    from lxml import etree
    xml_rules = etree.parse(open(file, 'r'))
    root = xml_rules.getroot()

    # first iteration to generate the dimacs variables
    for child in root :
        for subchild in child :
            for element in subchild : 
                if element.tag == 'name':
                    var_map.append(element.text)

    # generate variables
    for el in var_map:
        variables_string += "c " + str(1+var_map.index(el)) + " " + el + "\n"

    # generate rules
    for child in root :
        for subchild in child :
            for element in subchild :

                # temporary save current element and excluded options
                if element.tag == 'name':
                    current_options.append(element.text)
                if element.tag == 'excludedOptions' :
                    for option in element :
                        current_options.append(option.text)

                # clear temp if 'otional' tag is set to true
                if element.tag == 'optional' :
                    if element.text != 'False' :
                        current_options = []

            # if there are choices to do generate all combinations at once
            if len(current_options) > 0 :
                ###########################################################################
                # adding all combinations of rules if they are not already in 
                ###########################################################################
                # current_name, current_options
                #
                # print("Inside combinations loop", str(len(current_options)))

                if len(current_options) == 1 :
                    #print("exact one element")
                    for element in current_options:
                        #print("1")
                        tmp=[]
                        tmp.append(1+var_map.index(element))
                        clauses_2D.append(tmp)

                if len(current_options) > 1 :
                    ####################################################current_options = current_options.sort()
                    combinations = linear_regression(len(current_options))
                    for lst in combinations :
                        if not lst.count(False) == 1 :
                            tmp_list = []
                            for el, el2 in zip(lst, current_options) :
                                if el :
                                    tmp_list.append(1+var_map.index(el2))
                                else :
                                    tmp_list.append((1+var_map.index(el2))*(-1))

                            # do not put duplicate elements in list
                            tmp_list.sort()
                            is_in_list = False
                            for item in clauses_2D:
                                if item == tmp_list:
                                    is_in_list = True
                            if not is_in_list :
                                clauses_2D.append(tmp_list)

            current_options = []

    p_line += str(len(var_map))+" "+str(len(clauses_2D))+"\n"

    #print(clauses_2D)
    for line in clauses_2D :
        for item in line :
            clauses += str(item)+" "
        clauses += "0\n"

    # und zack fertig .... dimacs
    final_string = variables_string+p_line+clauses
    return final_string.split('\n')

In [4]:
class Wrapper :
    
    def __init__(self, path_to_folder, project_name) :
        from os import listdir
        from os.path import isfile, join
        
        
        self.fitness_model = Fitness_Modell()
        self.constraint_model = Constraint_Model()
        
        self.best = []
        self.popsize = 1000
        self.population = []
        self.fittness_list = []
       
        files = [join(path_to_folder, f) for f in listdir(path_to_folder) if isfile(join(path_to_folder, f))]
        tmp_dimacs = []
        
        for file in files:
             # -------- Fitness Model -----------
            if file.count(project_name) > 0 and (file.count('feature') > 0 or file.count('interactions') > 0) :
                with open(file, 'r') as f:
                    self.fitness_model.add_fitness_rules(f.readlines())
            
             # ------ Constraint Model ---------
            if file.count(project_name) > 0 and file.count('.xml') > 0 :
                tmp_dimacs = parse_xml_to_dimacs(file)
                self.constraint_model.build_model(tmp_dimacs)
            if file.count(project_name) > 0 and file.count('.dimacs') > 0 :
                with open(file, 'r') as f:
                    tmp_dimacs = f.readlines()
                    self.constraint_model.build_model(tmp_dimacs)                    
                    
                                    
        # analyze constraints
        self.constraint_model.build_global_tabu_list()
        
        # parse fittnes rule names to ints
        self.fitness_model.parse_names_to_index(tmp_dimacs)

            
    def init_population(self) :
        random.seed()
        for i in range(self.popsize) :
            self.population.append(self.init_individual())
            
        tmp_list = []
        for it in self.population :
            if not it == None :
                tmp_list.append(it)
                
        self.population = tmp_list
        self.popsize = len(self.population)
        
        for i in range(0, self.popsize) :
            self.fittness_list.append(self.fitness_model.calculate_fitness(self.population[i]))
        
    def init_population_pycosat(self) :
        self.population = pycosat_init(self.constraint_model, self.popsize)
            
    def do_steady_state(self) :
        alg = Algorithms()
        alg.steady_state(self, self.popsize)
        
    def init_individual(self) :     
        configuration = copy.deepcopy(self.constraint_model.mandatory_activation)
        tabu_config = copy.deepcopy(self.constraint_model.global_tabu_list)
        constraint_list = copy.deepcopy(self.constraint_model.constraint_list)
        
        is_valid = False
        
        counter = 0
        
        while not is_valid :
            for idx in range(len(configuration)) :
                if not tabu_config[idx]:
                    configuration[idx] = bool(random.getrandbits(1))
                    
            tmp = self.constraint_model.get_violated_variables(configuration)
            ptr = 0
            #print("length tmp: " + str(len(tmp)))
            for it in tmp :
                for i in range(ptr + 1, it - 1) :
                    tabu_config[i] = True
                tabu_config[it] = False
                ptr = it
            
            counter += 1
            
            if self.constraint_model.check_validity(configuration)  :
                is_valid = True
                print (counter)
                return configuration
            if counter > 10000 : 
                print (counter)
                return None
            
        return None

In [5]:
def pycosat_init(constraint_model, number_of_individuals) :
    import pycosat
    import itertools
    
    cnf = constraint_model.get_constraints()
    pop_list = []
    tmp_individual = []
    
    
    population = list(itertools.islice(pycosat.itersolve(cnf), number_of_individuals*2))
    for individual in population :
            
        # represents the root element
        tmp_individual.append(True)

        for element in individual :
            if (element > 0) :
                tmp_individual.append(True)
            else :
                tmp_individual.append(False)
        pop_list.append(tmp_individual)
        tmp_individual = []
                    
    #print(len(pop_list))
    return pop_list

In [50]:
class Algorithms :
    def __init__(self) :
        print("Initialize Algorithms Class")
        
        self.population = []
        self.fittness_of_population = []
        self.best = []
        self.best_fitness = None
        
        self.helper = Helper_Functions()
    
    def steady_state(self, wrapper, popsize_input) :
        import random
        print("Steady State")
        
        popsize = popsize_input
        
        #initialization
        self.population = []
        population_tmp = pycosat_init(wrapper.constraint_model, popsize)
        
        # appends elements, that are not already in p
        for ind in population_tmp :
            if not ind in self.population :
                self.population.append(ind)

        self.popsize = len(self.population)
        
        # assess fittness of all individuals
        for el in self.population :
            fitness_of_el = wrapper.fitness_model.calculate_fitness(el)
            self.fittness_of_population.append(fitness_of_el)
            if self.best_fitness == None or fitness_of_el < self.best_fitness :
                self.best = el
                self.best_fitness = fitness_of_el
        
        random.seed()
        
        # main algorithm
        not_done = True
        max_rounds = 1000
        counter = 0
        
        while not_done :
            
            # select two parents
            parent_a = self.tournament_selection(wrapper, self.population)
            parent_b = self.tournament_selection(wrapper, self.population)
            # print(len(parent_a), " ", len(parent_b))
            
            # does not produce every time a valid child - crossover two parents
            [child_a, child_b] = self.helper.crossover(parent_a, parent_b, wrapper.constraint_model)
            
            # Kann dann weg #######################################
            #print("Child1")
            #print(child_a)
            #print(wrapper.constraint_model.check_validity(child_a))
            #print(wrapper.fitness_model.calculate_fitness(child_a))
            #######################################################
            
            if (wrapper.constraint_model.check_validity(child_a)) :
                child_a = self.helper.mutate(child_a)
                
                fitness_child = wrapper.fitness_model.calculate_fitness(child_a)
                if fitness_child < self.best_fitness :
                    print("Child a is better as best")
                    self.best_fitness = fitness_child
                    self.best = child_a
                    
                self.population = self.helper.select_for_death(child_a, self.population)
                
            else :
                print ("child a is not valid")
                
            
            # Kann dann weg #######################################
            #print("Child2")
            #print(child_b)
            #print(wrapper.constraint_model.check_validity(child_b))
            #print(wrapper.fitness_model.calculate_fitness(child_b))
            #######################################################
            
            if (wrapper.constraint_model.check_validity(child_b)) :
                child_b = self.helper.mutate(child_b)
                
                fitness_child = wrapper.fitness_model.calculate_fitness(child_b)
                if fitness_child < self.best_fitness :
                    print("Child b is better as best")
                    self.best_fitness = fitness_child
                    self.best = child_b
                    
                self.population = self.helper.select_for_death(child_b, self.population)
                
            else :
                print ("child b is not valid")
            
            counter += 1
            print(counter)
            # maybe: or Best is optimum ?
            if counter > max_rounds :
                not_done = False
        
        
        #print(self.population)
        #print(self.fittness_of_population)
        print("Best:")
        #print(self.best)
        print(wrapper.fitness_model.calculate_fitness(self.best))
        
        
    def tournament_selection(self, wrapper, population) :
        p1 = population[random.randint(0, self.popsize-1)]
        p2 = population[random.randint(0, self.popsize-1)]
        
        if (wrapper.fitness_model.calculate_fitness(p1) < wrapper.fitness_model.calculate_fitness(p1)) :
            return p1
        else :
            return p2
        
    

In [51]:
class Helper_Functions :
    # This class contains needed functions to construct an machine learning algorithm
    def __init__(self) :
        self.mutations = 0
        
    def correct_violations(self, tweaked_individual, violated_constraint, constraint_model) :
        for it in violated_constraint :
            if not self.tabu[abs(it)] :
                tweaked_individual[abs(it)] = not tweaked_individual[abs(it)]
                self.tabu[abs(it)] = True
                violated_constraint_new = constraint_model.get_violated_constraint(tweaked_individual)
                if not violated_constraint_new == [] :
                    if not self.correct_violations(tweaked_individual, violated_constraint_new, constraint_model) :
                        tweaked_individual[abs(it)] = not tweaked_individual[abs(it)]
                    else :
                        self.mutations += 1
                        return True
                else :
                    self.mutations += 1
                    return True
        return False
        
    def tweak(self, individual, constraint_model) :
        # tries to tweak an individual by randomly modify its elements 
        # (we want to stay in the valid area, so check constraints)
        
        tweaked_individual = copy.deepcopy(individual)
        self.tabu = copy.deepcopy(constraint_model.global_tabu_list)
        
        is_valid = False
        counter = 0
        len_individual = len(individual)
        max_mutations = 0.05
        max_mutations = random.randint(0, int(max(max_mutations * len_individual, 1)))
        already_mutated = []
        
        counter = 0
        self.mutations = 0
        while self.mutations < max_mutations :
            gen_to_mutate = random.randint(1, len_individual)
            while tabu[gen_to_mutate] : 
                gen_to_mutate = random.randint(1, len_individual)
                counter += 1
                if counter > 1000 :
                    print("Tweak Failed at" + str(self.mutations) + "mutations")
                    return tweaked_individual
                
            tweaked_individual[gen_to_mutate] = not tweaked_individual[gen_to_mutate]
            self.tabu [gen_to_mutate] = True
            
            violated_constraint = constraint_model.get_violated_constraint(tweaked_individual)
            if not violated_constraint == [] :
                if not self.correct_violations(tweaked_individual, violated_constraint) :
                    tweaked_individual[gen_to_mutate] = not tweaked_individual[gen_to_mutate] 
                else :
                    self.mutations += 1      
                    
        return tweaked_individual
    
    
    def quality(self, individual, fitness_model) :
        # calgulates quality of an individual
        
        return fittness_model.calculate_fitness(individual)
        
    
    def crossover(self, parent1, parent2, constraint_model) :
        # generates two children by crossovering two parents
        
        self.tabu = copy.deepcopy(constraint_model.global_tabu_list)
        
        child1 = copy.deepcopy(parent1)
        child2 = copy.deepcopy(parent2)
        
        for i in range(1, len(child1)) :
            if (not child1[i] == child2[i]) and bool(random.getrandbits(1)) :
                child1[i] = not child1[i]
                child2[i] = not child2[i]
                
        self.tabu = copy.deepcopy(constraint_model.global_tabu_list)
        violated_constraint_c1 = constraint_model.get_violated_constraint(child1)
        if not violated_constraint_c1 == [] :
            if not self.correct_violations(child1, violated_constraint_c1, constraint_model) :
                child1[i] = not child1[i]

        self.tabu = copy.deepcopy(constraint_model.global_tabu_list)
        violated_constraint_c2 = constraint_model.get_violated_constraint(child1)
        if not violated_constraint_c2 == [] :
            if not self.correct_violations(child1, violated_constraint_c2, constraint_model) :
                child2[i] = not child2[i]
                
             
        return [child1, child2]
    
    
    def diversity(self, individual1, individual2) :
        # compares equality of two individuals element by element and return weighted sum
        
        if len(individual1) is not len(individual2) :
            print("size of both individuals should be equal to compare")
            return
        
        elements_equal = 0
        for el1, el2 in zip(individual1, individual2) :
            if el1 == el2 :
                elements_equal += 1

        return elements_equal/len(individual1)
    
    
    
    def evolution (self, wrapperinstanz, number_of_iterations) :
        random.seed()
        for i in range(0, number_of_iterations) :
            print("blub")
    
    def mutate (self, individual):
        # mutate a child to get diversity
        mutated_individual = copy.deepcopy(individual)
        return mutated_individual
    
    def select_for_death (self, individual, population) :
        # select one element of the population and replace it with the individual
        new_population = copy.deepcopy(population)
        return new_population

In [52]:
import random
import copy

#testinstanz = Wrapper('./project_public_2/', 'busy')
testinstanz = Wrapper('./project_public_2/', 'toy')
#testinstanz = Wrapper('./project_public_1/', 'bdbc')
#testinstanz = Wrapper('./project_public_1/', 'h264')

#testinstanz.init_population()
#testinstanz.init_population_pycosat()
testinstanz.do_steady_state()

Initialize Algorithms Class
Steady State
child b is not valid
1
child b is not valid
2
child b is not valid
3
child b is not valid
4
child b is not valid
5
child b is not valid
6
child b is not valid
7
child b is not valid
8
child b is not valid
9
child b is not valid
10
child b is not valid
11
child b is not valid
12
child b is not valid
13
14
child b is not valid
15
child b is not valid
16
17
child b is not valid
18
child b is not valid
19
child b is not valid
20
child b is not valid
21
child b is not valid
22
child b is not valid
23
child b is not valid
24
25
child b is not valid
26
child b is not valid
27
child b is not valid
28
child b is not valid
29
child b is not valid
30
child b is not valid
31
child b is not valid
32
child b is not valid
33
child b is not valid
34
child b is not valid
35
child b is not valid
36
child b is not valid
37
child b is not valid
38
child b is not valid
39
child b is not valid
40
child b is not valid
41
child b is not valid
42
child b is not valid
43

child b is not valid
346
child b is not valid
347
child b is not valid
348
child b is not valid
349
child b is not valid
350
child b is not valid
351
child b is not valid
352
child b is not valid
353
child b is not valid
354
child b is not valid
355
child b is not valid
356
child b is not valid
357
child b is not valid
358
child b is not valid
359
child b is not valid
360
child b is not valid
361
child b is not valid
362
child b is not valid
363
child b is not valid
364
child b is not valid
365
child b is not valid
366
child b is not valid
367
368
child b is not valid
369
child b is not valid
370
child b is not valid
371
child b is not valid
372
child b is not valid
373
child b is not valid
374
child b is not valid
375
child b is not valid
376
child b is not valid
377
child b is not valid
378
child b is not valid
379
child b is not valid
380
child b is not valid
381
382
child b is not valid
383
child b is not valid
384
child b is not valid
385
child b is not valid
386
child b is not va

child b is not valid
688
689
child b is not valid
690
child b is not valid
691
child b is not valid
692
child b is not valid
693
child b is not valid
694
child b is not valid
695
child b is not valid
696
child b is not valid
697
child b is not valid
698
child b is not valid
699
child b is not valid
700
701
child b is not valid
702
child b is not valid
703
child b is not valid
704
child b is not valid
705
child b is not valid
706
child b is not valid
707
child b is not valid
708
child b is not valid
709
child b is not valid
710
child b is not valid
711
child b is not valid
712
child b is not valid
713
child b is not valid
714
child b is not valid
715
child b is not valid
716
child b is not valid
717
child b is not valid
718
child b is not valid
719
child b is not valid
720
child b is not valid
721
child b is not valid
722
child b is not valid
723
child b is not valid
724
child b is not valid
725
child b is not valid
726
727
child b is not valid
728
child b is not valid
729
730
child b i